In [10]:
import pandas as pd, numpy as np, random, tensorflow as tf, keras_tuner
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, BatchNormalization, Activation, Flatten
from keras.layers.convolutional import Conv1D, AveragePooling1D, MaxPooling1D
from keras.optimizers import Adam

In [4]:
def data2series_trans(x):
    length = np.prod(x.shape)
    x_new = x.to_numpy().reshape(length, 1)
    y_0 = (x_new/10).astype(int)
    y_1 = (x_new%10).astype(int)
    return x_new/38, y_0, y_1

def prepare2data(x_new, y_0, y_1, lag):
    n = int(x_new.shape[0]/7) - lag
    x = [x_new[i*7:(i+lag)*7] for i in range(n)]
    y0 = [y_0[(i+lag)*7:(i+1+lag)*7] for i in range(n)]
    y1 = [y_1[(i+lag)*7:(i+1+lag)*7] for i in range(n)]
    x_need = x_new[n*7:x_new.shape[0]][np.newaxis, :]
    return np.array(x), np.array(y0), np.array(y1), x_need

In [13]:
def hyper_para(model, X_tra, y0_tra, y1_tra, X_val, y0_val, y1_val, X_test, y0_test, y1_test, X_need):
            
    tuner = keras_tuner.Hyperband(hypermodel = model, objective = "val_loss", max_epochs = 20, factor = 3,
                                  overwrite = True, directory = "my_dir", project_name = "testing1234")
    tuner.search_space_summary() 
    stop_early = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 10)
    tuner.search(X_tra, [y0_tra, y1_tra], epochs = 100, batch_size = 64, validation_data = (X_val, [y0_val, y1_val]), 
                 callbacks = [stop_early])
    best_hp = tuner.get_best_hyperparameters()[0]
            
    h_model = tuner.hypermodel.build(best_hp)
    h_model.summary()
    history = h_model.fit(X_tra, [y0_tra, y1_tra], epochs = 100, batch_size = 64, validation_data = (X_val, [y0_val, y1_val]), 
                          callbacks = [stop_early])
            
    val_acc_per_epoch = history.history["val_loss"]
    best_epoch = val_acc_per_epoch.index(min(val_acc_per_epoch)) + 1
    model = tuner.hypermodel.build(best_hp)
    h_model.fit(X_tra, [y0_tra, y1_tra], epochs = best_epoch, batch_size = 32, validation_data = (X_val, [y0_val, y1_val]), 
                callbacks = [stop_early])
    h_eval_dict = h_model.evaluate(X_test, [y0_test, y1_test], return_dict = True) 
    y_pred_hat = h_model.predict(X_need)
    return h_eval_dict, y_pred_hat

In [11]:
def nn_model(lag, seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.keras.utils.set_random_seed(seed)
    tf.config.experimental.enable_op_determinism()
    
    x_new, y_0, y_1 = data2series_trans(power.iloc[:, 1:])
    X, y0, y1, X_need = prepare2data(x_new, y_0, y_1, lag)
    
    m = int(X.shape[0]*0.9)
    
    X_train = X[0:m]
    y0_train = y0[0:m]
    y1_train = y1[0:m]
    
    X_test = X[m:X.shape[0]]
    y0_test = y0[m:X.shape[0]]
    y1_test = y1[m:X.shape[0]]
    
    n = int(X_train.shape[0]*0.8)
    
    X_val = X_train[n:m]
    y0_val = y0_train[n:m]
    y1_val = y1_train[n:m]
    
    X_tra = X_train[0:n]
    y0_tra = y0_train[0:n]
    y1_tra = y1_train[0:n]
    
    n_steps_in = X_tra.shape[1]
    n_features = 1
    label_dim = 7

    def model_set(hp):
        inp = Input((n_steps_in, n_features))
        s = Conv1D(filters = hp.Int("filters", min_value = 900, max_value = 1500, step = 20), kernel_size = 7)(inp)
        s = Activation("relu")(s)
        s = BatchNormalization()(s)
        s = MaxPooling1D(pool_size = 7)(s)
        s = Dropout(rate = hp.Float("rate", min_value = 0.1, max_value = 0.5))(s)
        s = Conv1D(filters = hp.Int("filters", min_value = 300, max_value = 900, step = 20), kernel_size = 7)(s)
        s = Activation("relu")(s)
        s = BatchNormalization()(s)
        s = MaxPooling1D(pool_size = 7)(s)
        s = Dropout(rate = hp.Float("rate", min_value = 0.1, max_value = 0.5))(s)
        s = Flatten()(s)
        s = Dense(units = 30, activation = "relu")(s)
        s = BatchNormalization()(s)
        s = Dropout(rate = hp.Float("rate", min_value = 0.1, max_value = 0.5))(s)
        out1 = Dense(label_dim)(s)
        out2 = Dense(label_dim)(s)
        model = Model(inp, [out1, out2])
        adam = Adam(learning_rate = 0.0001)
        model.compile(loss = "mean_squared_logarithmic_error", optimizer = adam)
        return model

    results = hyper_para(model_set, X_tra, y0_tra, y1_tra, X_val, y0_val, y1_val, X_test, y0_test, y1_test, X_need)
    
    return results

In [8]:
power = "insert your data"

In [14]:
results = nn_model(28, 1111)

Trial 30 Complete [00h 00m 11s]
val_loss: 24.2031192779541

Best val_loss So Far: 23.358152389526367
Total elapsed time: 00h 02m 06s
INFO:tensorflow:Oracle triggered exit
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 196, 1)]     0           []                               
                                                                                                  
 conv1d_2 (Conv1D)              (None, 190, 920)     7360        ['input_2[0][0]']                
                                                                                                  
 activation_2 (Activation)      (None, 190, 920)     0           ['conv1d_2[0][0]']               
                                                                                                  
 batch_normalization

2023-07-25 14:01:34.829041: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype int64 and shape [1476,7,1]
	 [[{{node Placeholder/_12}}]]
2023-07-25 14:01:34.829285: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype int64 and shape [1476,7,1]
	 [[{{node Placeholder/_12}}]]


24/24 [==============================] - 1s 19ms/step - loss: 29.5866 - dense_4_loss: 27.5184 - dense_5_loss: 2.0682 - val_loss: 28.5367 - val_dense_4_loss: 26.4393 - val_dense_5_loss: 2.0975
Epoch 2/100
 1/24 [>.............................] - ETA: 0s - loss: 28.0160 - dense_4_loss: 25.9837 - dense_5_loss: 2.0323

2023-07-25 14:01:35.720247: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype int64 and shape [369,7,1]
	 [[{{node Placeholder/_12}}]]
2023-07-25 14:01:35.720449: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype int64 and shape [369,7,1]
	 [[{{node Placeholder/_12}}]]


24/24 [==============================] - 0s 13ms/step - loss: 27.4082 - dense_4_loss: 25.3650 - dense_5_loss: 2.0432 - val_loss: 24.8684 - val_dense_4_loss: 22.4918 - val_dense_5_loss: 2.3766
Epoch 3/100
24/24 [==============================] - 0s 13ms/step - loss: 26.4642 - dense_4_loss: 24.4310 - dense_5_loss: 2.0332 - val_loss: 23.7949 - val_dense_4_loss: 21.7748 - val_dense_5_loss: 2.0201
Epoch 4/100
24/24 [==============================] - 0s 13ms/step - loss: 26.1132 - dense_4_loss: 24.1020 - dense_5_loss: 2.0113 - val_loss: 23.4541 - val_dense_4_loss: 21.8890 - val_dense_5_loss: 1.5651
Epoch 5/100
24/24 [==============================] - 0s 13ms/step - loss: 25.9130 - dense_4_loss: 23.9147 - dense_5_loss: 1.9982 - val_loss: 23.6045 - val_dense_4_loss: 22.1234 - val_dense_5_loss: 1.4811
Epoch 6/100
24/24 [==============================] - 0s 12ms/step - loss: 25.7420 - dense_4_loss: 23.7611 - dense_5_loss: 1.9809 - val_loss: 24.0788 - val_dense_4_loss: 22.4545 - val_dense_5_loss:

2023-07-25 14:01:39.964378: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [1476,196,1]
	 [[{{node Placeholder/_10}}]]
2023-07-25 14:01:39.964586: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [1476,7,1]
	 [[{{node Placeholder/_11}}]]


47/47 [==============================] - 0s 10ms/step - loss: 24.8970 - dense_4_loss: 23.0893 - dense_5_loss: 1.8078 - val_loss: 24.0559 - val_dense_4_loss: 22.4201 - val_dense_5_loss: 1.6358
Epoch 2/4
13/47 [=======>......................] - ETA: 0s - loss: 25.1207 - dense_4_loss: 23.3218 - dense_5_loss: 1.7989

2023-07-25 14:01:40.414013: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [369,7,1]
	 [[{{node Placeholder/_11}}]]
2023-07-25 14:01:40.414228: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype int64 and shape [369,7,1]
	 [[{{node Placeholder/_12}}]]


47/47 [==============================] - 0s 9ms/step - loss: 24.7870 - dense_4_loss: 23.0063 - dense_5_loss: 1.7807 - val_loss: 24.0077 - val_dense_4_loss: 22.4157 - val_dense_5_loss: 1.5920
Epoch 3/4
47/47 [==============================] - 0s 9ms/step - loss: 24.6875 - dense_4_loss: 22.9294 - dense_5_loss: 1.7581 - val_loss: 23.8482 - val_dense_4_loss: 22.2314 - val_dense_5_loss: 1.6169
Epoch 4/4
1/1 [==============================] - 0s 125ms/step


2023-07-25 14:01:41.849996: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype int64 and shape [206,7,1]
	 [[{{node Placeholder/_12}}]]
2023-07-25 14:01:41.850277: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype int64 and shape [206,7,1]
	 [[{{node Placeholder/_12}}]]
2023-07-25 14:01:41.922509: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and sha

In [15]:
print(results)

({'loss': 23.8367862701416, 'dense_4_loss': 22.26786994934082, 'dense_5_loss': 1.5689131021499634}, [array([[ 0.7279916,  9.381762 , -5.0315046, -1.6490586,  0.9666617,
        -3.3279016, -6.2033725]], dtype=float32), array([[ 0.81983334,  3.0957782 ,  3.863658  , -3.4761794 ,  1.8033481 ,
         2.2087786 , -3.8284883 ]], dtype=float32)])
